In [1]:
import os
import numpy as np
import pandas as pd

dataPath = (os.getcwd().replace("\\", "/")) + "/../자료/"
nonSharedDataPath = os.getcwd().replace("\\", "/") + "/../../공유못함/"

In [62]:
pTDF = pd.read_csv(nonSharedDataPath + '행정동별대중교통수단통합.csv', sep=',', encoding='euc-kr')
pTDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27560766 entries, 0 to 27560765
Data columns (total 6 columns):
 #   Column     Dtype
---  ------     -----
 0   기준_날짜      int64
 1   시작_행정동_ID  int64
 2   종료_행정동_ID  int64
 3   승객수        int64
 4   시작_행정구_ID  int64
 5   종료_행정구_ID  int64
dtypes: int64(6)
memory usage: 1.2 GB


In [6]:
mappingDF = pd.read_csv(dataPath + '행정동매핑.csv', encoding='euc-kr')
mappingDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426 entries, 0 to 425
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   행정구코드   426 non-null    int64 
 1   시군구     426 non-null    object
 2   행정동코드   426 non-null    int64 
 3   읍면동     426 non-null    object
dtypes: int64(2), object(2)
memory usage: 13.4+ KB


In [64]:
c1 = (pTDF['기준_날짜'] == 0)
c2 = (pTDF['시작_행정동_ID'] == 0)
c3 = (pTDF['종료_행정동_ID'] == 0)
(c1 | c2 | c3).unique()
pTDF = pTDF[(c1 | c2 | c3) == False]

In [65]:
def getGuID(x):
    x[4] = int(x[1] / 100)
    x[5] = int(x[2] / 100)
    
    return x

checkDF = pTDF[:5]
checkDF['시작_행정구_ID'] = ""
checkDF['종료_행정구_ID'] = ""

checkDF = pd.DataFrame(np.apply_along_axis(getGuID, 1, checkDF), columns=checkDF.columns.to_list())
checkDF
# checkDF.apply(getGuID, axis=1)

C:\Users\bigdata\AppData\Local\Temp\ipykernel_11704\855479904.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checkDF['시작_행정구_ID'] = ""
C:\Users\bigdata\AppData\Local\Temp\ipykernel_11704\855479904.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checkDF['종료_행정구_ID'] = ""


,기준_날짜,시작_행정동_ID,종료_행정동_ID,승객수,시작_행정구_ID,종료_행정구_ID
0,20220101,1112056,2305057,6,11120,23050
1,20220101,1113071,1113070,46,11130,11130
2,20220101,1102071,1125074,8,11020,11250
3,20220101,1105053,1110065,149,11050,11100
4,20220101,1104052,1102065,43,11040,11020


In [66]:
pTDF['시작_행정구_ID'] = ""
pTDF['종료_행정구_ID'] = ""

def getGuID(x):
    x[4] = int(x[1] / 100)
    x[5] = int(x[2] / 100)
    
    return x

pTDF2 = pd.DataFrame(np.apply_along_axis(getGuID, 1, pTDF), columns=pTDF.columns.to_list())
#pTDF.apply(getGuID, axis=1)
pTDF2

,기준_날짜,시작_행정동_ID,종료_행정동_ID,승객수,시작_행정구_ID,종료_행정구_ID
0,20220101,1112056,2305057,6,11120,23050
1,20220101,1113071,1113070,46,11130,11130
2,20220101,1102071,1125074,8,11020,11250
3,20220101,1105053,1110065,149,11050,11100
4,20220101,1104052,1102065,43,11040,11020
...,...,...,...,...,...,...
27560761,20220430,3102154,1104066,8,31021,11040
27560762,20220430,1112074,1115054,1,11120,11150
27560763,20220430,2308060,1123066,3,23080,11230
27560764,20220430,1123052,1123052,966,11230,11230


In [ ]:
pTDF2.to_csv(nonSharedDataPath + '행정동별대중교통수단통합.csv', encoding='euc-kr', index=False)

In [ ]:
# 자치구별 대중교통현황 자료 만들기

pTDF2 = pd.read_csv(nonSharedDataPath + '행정동별대중교통수단통합.csv', encoding='euc-kr')
pTDF2[pTDF2.columns[:3].append(pTDF2.columns[4:])] = pTDF2[pTDF2.columns[:3].append(pTDF2.columns[4:])].astype('str')
pTDF2.info()

In [ ]:
shGroup = pTDF2.groupby(pTDF2.columns[4:].append(pTDF2.columns[0:1]).to_list(), as_index=False).agg('sum').sort_values('승객수', ascending=False)
shGroup.info()

In [ ]:
shGroupStart = shGroup.groupby(['시작_행정구_ID', '기준_날짜'], as_index=False).agg('sum')
shGroupStart.rename({'시작_행정구_ID' : '행정구'}, inplace=True, axis=1)
shGroupStart

In [ ]:
shGroupEnd = shGroup.groupby(['종료_행정구_ID', '기준_날짜'], as_index=False).agg('sum')
shGroupEnd.rename({'종료_행정구_ID' : '행정구'}, inplace=True, axis=1)
shGroupEnd

In [ ]:
sumDF = shGroupStart.merge(shGroupEnd, on=['행정구', '기준_날짜'])
sumDF['승객수'] = ((sumDF['승객수_x'] + sumDF['승객수_y']) / 2).astype('int')
sumDF['행정구'] = sumDF['행정구'].astype('int')
sumDF = sumDF[['행정구', '승객수', '기준_날짜']][sumDF['행정구'] < 20000]

In [ ]:
print(sumDF.head(1))
print(mappingDF.head(1))

In [ ]:
sumDF['시군구'] = ""

def mappingGu(x):
    for i in mappingDF.index:
        if (x[0] == mappingDF.iloc[i, 0]):
            x[3] = mappingDF.iloc[i, 1]
            break;
    return x
            
sumDF = pd.DataFrame(np.apply_along_axis(mappingGu, 1, sumDF), columns=sumDF.columns)
sumDF

In [ ]:
sumDF.to_csv(dataPath + '대중교통자치구별현황.csv', encoding='euc-kr', index=False)

In [61]:
#행정동별 대중교통현황 자료 만들기

pTDF2 = pd.read_csv(nonSharedDataPath + '행정동별대중교통수단통합.csv', encoding='euc-kr')
pTDF2[pTDF2.columns[:3].append(pTDF2.columns[4:])] = pTDF2[pTDF2.columns[:3].append(pTDF2.columns[4:])].astype('str')
pTDF2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27560766 entries, 0 to 27560765
Data columns (total 6 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   기준_날짜      object
 1   시작_행정동_ID  object
 2   종료_행정동_ID  object
 3   승객수        int64 
 4   시작_행정구_ID  object
 5   종료_행정구_ID  object
dtypes: int64(1), object(5)
memory usage: 1.2+ GB


In [62]:
shGroupStart = pTDF2.groupby(['시작_행정동_ID', '기준_날짜', '시작_행정구_ID'], as_index=False).agg('sum')
shGroupStart.rename({'시작_행정동_ID' : '행정동', '시작_행정구_ID' : '시군구'}, inplace=True, axis=1)
shGroupStart

,행정동,기준_날짜,시군구,승객수
0,1101053,20220101,11010,20161
1,1101053,20220102,11010,21299
2,1101053,20220103,11010,53671
3,1101053,20220104,11010,56770
4,1101053,20220105,11010,57669
...,...,...,...,...
84425,4413358,20220426,44133,95
84426,4413358,20220427,44133,1023
84427,4413358,20220428,44133,1097
84428,4413358,20220429,44133,1090


In [63]:
shGroupEnd = pTDF2.groupby(['종료_행정동_ID', '기준_날짜', '종료_행정구_ID'], as_index=False).agg('sum')
shGroupEnd.rename({'종료_행정동_ID' : '행정동', '종료_행정구_ID' : '시군구'}, inplace=True, axis=1)
shGroupEnd

,행정동,기준_날짜,시군구,승객수
0,1101053,20220101,11010,20888
1,1101053,20220102,11010,22103
2,1101053,20220103,11010,54991
3,1101053,20220104,11010,58251
4,1101053,20220105,11010,59025
...,...,...,...,...
84403,4413358,20220426,44133,57
84404,4413358,20220427,44133,929
84405,4413358,20220428,44133,991
84406,4413358,20220429,44133,887


In [64]:
sumDF = shGroupStart.merge(shGroupEnd, on=['행정동', '기준_날짜', '시군구'])
sumDF['승객수'] = ((sumDF['승객수_x'] + sumDF['승객수_y']) / 2).astype('int')
sumDF['행정동'] = sumDF['행정동'].astype('int')
sumDF['시군구'] = sumDF['시군구'].astype('int')
sumDF = sumDF[['시군구', '행정동', '승객수', '기준_날짜']][sumDF['시군구'] < 20000]

In [65]:
print(sumDF.head(1))
print(mappingDF.head(1))

     시군구      행정동    승객수     기준_날짜
0  11010  1101053  20524  20220101
   행정구코드  시군구    행정동코드  읍면동
0  11010  종로구  1101053  사직동


In [69]:
sumDF['시군구명칭'] = ""
sumDF['행정동명칭'] = ""

def mappingGu(x):
    for i in mappingDF.index:
        if (x[1] == mappingDF.iloc[i, 2]):
            x[4] = mappingDF.iloc[i, 1]
            x[5] = mappingDF.iloc[i, 3]
            break;
    return x

tempDF = pd.DataFrame(np.apply_along_axis(mappingGu, 1, sumDF.head(300)), columns=sumDF.columns)
tempDF

,시군구,행정동,승객수,기준_날짜,시군구명칭,행정동명칭
0,11010,1101053,20524,20220101,종로구,사직동
1,11010,1101053,21701,20220102,종로구,사직동
2,11010,1101053,54331,20220103,종로구,사직동
3,11010,1101053,57510,20220104,종로구,사직동
4,11010,1101053,58347,20220105,종로구,사직동
...,...,...,...,...,...,...
295,11010,1101055,7925,20220324,종로구,부암동
296,11010,1101055,7033,20220325,종로구,부암동
297,11010,1101055,3626,20220326,종로구,부암동
298,11010,1101055,3377,20220327,종로구,부암동


In [70]:
sumDF = pd.DataFrame(np.apply_along_axis(mappingGu, 1, sumDF), columns=sumDF.columns)
sumDF

,시군구,행정동,승객수,기준_날짜,시군구명칭,행정동명칭
0,11010,1101053,20524,20220101,종로구,사직동
1,11010,1101053,21701,20220102,종로구,사직동
2,11010,1101053,54331,20220103,종로구,사직동
3,11010,1101053,57510,20220104,종로구,사직동
4,11010,1101053,58347,20220105,종로구,사직동
...,...,...,...,...,...,...
47058,11250,1125074,1183,20220426,강동구,길동
47059,11250,1125074,16513,20220427,강동구,길동
47060,11250,1125074,16654,20220428,강동구,길동
47061,11250,1125074,17058,20220429,강동구,길동


In [71]:
sumDF.to_csv(dataPath + '대중교통행정동별현황.csv', encoding='euc-kr', index=False)

In [15]:
#행정동별 교통량현황 자료 만들기

carMovingDF = pd.read_csv(dataPath + '행정동추가.csv', sep=',', encoding='euc-kr')
carMovingDF = carMovingDF[['지점명칭', '행정동']]
carMovingDF.rename({'지점명칭' : '지점명'}, axis=1, inplace=True)

carMovingSumDF = pd.read_csv(dataPath + '통행합계.csv', sep=',', encoding='euc-kr')
carMovingSumDF = carMovingSumDF[['일자', '지점명', '통행량']]

In [16]:
carMovingSumDF = carMovingSumDF.merge(carMovingDF, on='지점명')
carMovingSumDF

,일자,지점명,통행량,행정동
0,20220101,성산로(금화터널),0,신촌동
1,20220102,성산로(금화터널),0,신촌동
2,20220103,성산로(금화터널),0,신촌동
3,20220104,성산로(금화터널),0,신촌동
4,20220105,성산로(금화터널),0,신촌동
...,...,...,...,...
33355,20220426,신월여의지하도로,47150,신정동
33356,20220427,신월여의지하도로,47572,신정동
33357,20220428,신월여의지하도로,49836,신정동
33358,20220429,신월여의지하도로,49690,신정동


In [17]:
carMovingSumDF.rename({'행정동' : '읍면동'}, axis=1, inplace=True)
carMovingSumDF = carMovingSumDF[['일자', '통행량', '읍면동']]
carMovingSumDF

,일자,통행량,읍면동
0,20220101,0,신촌동
1,20220102,0,신촌동
2,20220103,0,신촌동
3,20220104,0,신촌동
4,20220105,0,신촌동
...,...,...,...
33355,20220426,47150,신정동
33356,20220427,47572,신정동
33357,20220428,49836,신정동
33358,20220429,49690,신정동


In [18]:
def check(x):
    if x[:2] == '신정': 
        return True 
    return False

mappingDF['읍면동'].apply(check).value_counts()

False    420
True       6
Name: 읍면동, dtype: int64

In [53]:
carGroup = carMovingSumDF.groupby(['일자', '읍면동'], as_index=False).agg('sum')
carGroup['읍면동'][carGroup['읍면동'] == '개화동'] = '방화동'
carGroup['통행량'] = carGroup['통행량'] / 2
carGroup

C:\Users\bigdata\AppData\Local\Temp\ipykernel_1820\2648121764.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  carGroup['읍면동'][carGroup['읍면동'] == '개화동'] = '방화동'


,일자,읍면동,통행량
0,20220101,가산동,175685.0
1,20220101,감북동,32999.0
2,20220101,개봉동,55845.0
3,20220101,방화동,210029.0
4,20220101,경운동,17872.0
...,...,...,...
12235,20220430,행촌동,61427.0
12236,20220430,홍은동,69647.0
12237,20220430,화곡동,38330.0
12238,20220430,황학동,38329.0


In [54]:
mappingDF.info()
carGroup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426 entries, 0 to 425
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   행정구코드   426 non-null    int64 
 1   시군구     426 non-null    object
 2   행정동코드   426 non-null    int64 
 3   읍면동     426 non-null    object
dtypes: int64(2), object(2)
memory usage: 13.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12240 entries, 0 to 12239
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   일자      12240 non-null  int64  
 1   읍면동     12240 non-null  object 
 2   통행량     12240 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 287.0+ KB


In [55]:
carGroup['행정동코드'] = ""
carGroup['시군구'] = ""
carGroup['행정구코드'] = ""

def mappingDong(x):
    for i in mappingDF.index:
        if(x[1][:2] == mappingDF.iloc[i, 3][:2]):
            x[3] = mappingDF.iloc[i, 1]
            x[4] = mappingDF.iloc[i, 2]
            x[5] = mappingDF.iloc[i, 0]
            break
            
    return x

In [56]:
tempDF = carGroup.head(5)
tempDF = pd.DataFrame(np.apply_along_axis(mappingDong, 1, tempDF), columns=carGroup.columns)
tempDF

,일자,읍면동,통행량,행정동코드,시군구,행정구코드
0,20220101,가산동,175685.0,금천구,1118051,11180
1,20220101,감북동,32999.0,,,
2,20220101,개봉동,55845.0,구로구,1117064,11170
3,20220101,방화동,210029.0,강서구,1116070,11160
4,20220101,경운동,17872.0,,,


In [72]:
carGroupDF = pd.DataFrame(np.apply_along_axis(mappingDong, 1, carGroup), columns=carGroup.columns)
carGroupDF

,일자,읍면동,통행량,행정동코드,시군구,행정구코드
0,20220101,가산동,175685.0,금천구,1118051,11180
2,20220101,개봉동,55845.0,구로구,1117064,11170
3,20220101,방화동,210029.0,강서구,1116070,11160
5,20220101,고덕동,83705.0,강동구,1125055,11250
7,20220101,공릉동,92723.0,노원구,1111056,11110
...,...,...,...,...,...,...
12234,20220430,합정동,131216.0,마포구,1114068,11140
12236,20220430,홍은동,69647.0,서대문구,1113066,11130
12237,20220430,화곡동,38330.0,강서구,1116055,11160
12238,20220430,황학동,38329.0,중구,1102067,11020


In [73]:
carGroupDF = carGroupDF[carGroupDF['행정동코드'] != ""]
carGroupDF['통행량'] = carGroupDF['통행량'].astype('int')
carGroupDF.info()
carGroupDF

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8880 entries, 0 to 12239
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   일자      8880 non-null   object
 1   읍면동     8880 non-null   object
 2   통행량     8880 non-null   int32 
 3   행정동코드   8880 non-null   object
 4   시군구     8880 non-null   object
 5   행정구코드   8880 non-null   object
dtypes: int32(1), object(5)
memory usage: 450.9+ KB


,일자,읍면동,통행량,행정동코드,시군구,행정구코드
0,20220101,가산동,175685,금천구,1118051,11180
2,20220101,개봉동,55845,구로구,1117064,11170
3,20220101,방화동,210029,강서구,1116070,11160
5,20220101,고덕동,83705,강동구,1125055,11250
7,20220101,공릉동,92723,노원구,1111056,11110
...,...,...,...,...,...,...
12234,20220430,합정동,131216,마포구,1114068,11140
12236,20220430,홍은동,69647,서대문구,1113066,11130
12237,20220430,화곡동,38330,강서구,1116055,11160
12238,20220430,황학동,38329,중구,1102067,11020


In [74]:
carGroupDF.to_csv(dataPath + '행정동별통행량현황.csv', encoding='euc-kr', index=False)

In [85]:
#자동차 현황 매핑

carHavingDF = pd.read_excel(dataPath + 'car_total.xlsx')
carHavingDF

,행정동사용본거지,행정동사용본거지코드,차종종별,연료,건수,비율
0,서울특별시 영등포구 여의동,1156054000,승용,경유,36691,0.168009
1,서울특별시 영등포구 여의동,1156054000,승용,휘발유(무연),34512,0.158031
2,서울특별시 영등포구 여의동,1156054000,승용,휘발유,33805,0.154794
3,서울특별시 강남구 역삼1동,1168064000,승용,휘발유(무연),28889,0.132283
4,서울특별시 강서구 가양1동,1150060300,승용,휘발유(무연),28016,0.128286
...,...,...,...,...,...,...
19216,서울특별시 동대문구 청량리동,1123070500,승용,휘발유(유연),1,0.000005
19217,서울특별시 동대문구 청량리동,1123070500,승합,기타연료,1,0.000005
19218,서울특별시 은평구 불광2동,1138053000,승용,하이브리드(경유+전기),1,0.000005
19219,서울특별시 강남구 일원1동,1168073000,특수,휘발유,1,0.000005


In [86]:
def getDong(x):
    tempList = x[0].split(' ')
    x[1] = tempList[2]
    return x

carHavingDF = pd.DataFrame(np.apply_along_axis(getDong, 1, carHavingDF), columns=carHavingDF.columns)
carHavingDF

,행정동사용본거지,행정동사용본거지코드,차종종별,연료,건수,비율
0,서울특별시 영등포구 여의동,여의동,승용,경유,36691,0.168009
1,서울특별시 영등포구 여의동,여의동,승용,휘발유(무연),34512,0.158031
2,서울특별시 영등포구 여의동,여의동,승용,휘발유,33805,0.154794
3,서울특별시 강남구 역삼1동,역삼1동,승용,휘발유(무연),28889,0.132283
4,서울특별시 강서구 가양1동,가양1동,승용,휘발유(무연),28016,0.128286
...,...,...,...,...,...,...
19216,서울특별시 동대문구 청량리동,청량리동,승용,휘발유(유연),1,0.000005
19217,서울특별시 동대문구 청량리동,청량리동,승합,기타연료,1,0.000005
19218,서울특별시 은평구 불광2동,불광2동,승용,하이브리드(경유+전기),1,0.000005
19219,서울특별시 강남구 일원1동,일원1동,특수,휘발유,1,0.000005


In [87]:
carHavingDF.rename({'행정동사용본거지코드' : '읍면동'}, axis=1, inplace=True)
carHavingDF = carHavingDF[carHavingDF.columns[1:]]
carHavingDF

,읍면동,차종종별,연료,건수,비율
0,여의동,승용,경유,36691,0.168009
1,여의동,승용,휘발유(무연),34512,0.158031
2,여의동,승용,휘발유,33805,0.154794
3,역삼1동,승용,휘발유(무연),28889,0.132283
4,가양1동,승용,휘발유(무연),28016,0.128286
...,...,...,...,...,...
19216,청량리동,승용,휘발유(유연),1,0.000005
19217,청량리동,승합,기타연료,1,0.000005
19218,불광2동,승용,하이브리드(경유+전기),1,0.000005
19219,일원1동,특수,휘발유,1,0.000005


In [88]:
mappingDF.info()
carHavingDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426 entries, 0 to 425
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   행정구코드   426 non-null    int64 
 1   시군구     426 non-null    object
 2   행정동코드   426 non-null    int64 
 3   읍면동     426 non-null    object
dtypes: int64(2), object(2)
memory usage: 13.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19221 entries, 0 to 19220
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   읍면동     19221 non-null  object
 1   차종종별    19221 non-null  object
 2   연료      19221 non-null  object
 3   건수      19221 non-null  object
 4   비율      19221 non-null  object
dtypes: object(5)
memory usage: 750.9+ KB


In [89]:
carHavingDF['시군구'] = ""
carHavingDF['행정동코드'] = ""
carHavingDF['행정구코드'] = ""

def changeDong(x):
    for i in mappingDF.index:
        if (x[0][:2] == mappingDF.iloc[i, 3][:2]):
            x[0] = mappingDF.iloc[i, 3]
            x[5] = mappingDF.iloc[i, 1]
            x[6] = mappingDF.iloc[i, 2]
            x[7] = mappingDF.iloc[i, 0]
            break;
            
    return x

tempDF = pd.DataFrame(np.apply_along_axis(changeDong, 1, carHavingDF.head()), columns=carHavingDF.columns)
tempDF

,읍면동,차종종별,연료,건수,비율,시군구,행정동코드,행정구코드
0,여의동,승용,경유,36691,0.168009,영등포구,1119054,11190
1,여의동,승용,휘발유(무연),34512,0.158031,영등포구,1119054,11190
2,여의동,승용,휘발유,33805,0.154794,영등포구,1119054,11190
3,역삼1동,승용,휘발유(무연),28889,0.132283,강남구,1123064,11230
4,가양1동,승용,휘발유(무연),28016,0.128286,강서구,1116064,11160


In [91]:
carHavingDF = pd.DataFrame(np.apply_along_axis(changeDong, 1, carHavingDF), columns=carHavingDF.columns)
carHavingDF

,읍면동,차종종별,연료,건수,비율,시군구,행정동코드,행정구코드
0,여의동,승용,경유,36691,0.168009,영등포구,1119054,11190
1,여의동,승용,휘발유(무연),34512,0.158031,영등포구,1119054,11190
2,여의동,승용,휘발유,33805,0.154794,영등포구,1119054,11190
3,역삼1동,승용,휘발유(무연),28889,0.132283,강남구,1123064,11230
4,가양1동,승용,휘발유(무연),28016,0.128286,강서구,1116064,11160
...,...,...,...,...,...,...,...,...
19216,청량리동,승용,휘발유(유연),1,0.000005,동대문구,1106080,11060
19217,청량리동,승합,기타연료,1,0.000005,동대문구,1106080,11060
19218,불광1동,승용,하이브리드(경유+전기),1,0.000005,은평구,1112052,11120
19219,일원본동,특수,휘발유,1,0.000005,강남구,1123072,11230


In [96]:
carHavingDF = carHavingDF.reindex(columns=['차종종별', '연료', '건수', '비율', '읍면동', '행정동코드', '시군구', '행정구코드'])
carHavingDF

,차종종별,연료,건수,비율,읍면동,행정동코드,시군구,행정구코드
0,승용,경유,36691,0.168009,여의동,1119054,영등포구,11190
1,승용,휘발유(무연),34512,0.158031,여의동,1119054,영등포구,11190
2,승용,휘발유,33805,0.154794,여의동,1119054,영등포구,11190
3,승용,휘발유(무연),28889,0.132283,역삼1동,1123064,강남구,11230
4,승용,휘발유(무연),28016,0.128286,가양1동,1116064,강서구,11160
...,...,...,...,...,...,...,...,...
19216,승용,휘발유(유연),1,0.000005,청량리동,1106080,동대문구,11060
19217,승합,기타연료,1,0.000005,청량리동,1106080,동대문구,11060
19218,승용,하이브리드(경유+전기),1,0.000005,불광1동,1112052,은평구,11120
19219,특수,휘발유,1,0.000005,일원본동,1123072,강남구,11230


In [97]:
carHavingDF.to_csv(dataPath + '행정동별차량현황.csv', encoding='euc-kr', index=False)